In [1]:
%matplotlib notebook

import os
import datetime as dt
import pickle, joblib


# Standard data science libraries
import pandas as pd
import numpy as np
import scipy.stats as ss
import scipy.optimize as so 
import scipy.interpolate as si

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-notebook')

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 200

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.display import Image
from IPython.display import Math



In [2]:
from ipywidgets import interact, Dropdown
from IPython.display import display

import flopy as fp
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point

import RTD_util6 as rtd_ut
import Genmod_Utilities as gmu

import matplotlib.dates as mdates
import matplotlib.ticker as mticks


In [3]:
KS1 = '#06366E'
KS2 = '#00A3EB'
KS3 = '#25C0A6'
KS4 = '#FDDA58'
KS5 = '#5D171A'

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12,
        'sans-serif' : 'Arial'}

plt.rc('font', **font)

In [4]:
total_number_of_particles = 4.E+06
por = 0.20
mp_exe_name7 = '../Executables/modpath_7_2_001/bin/mpath7.exe'

with open('GenMod_metadata.txt') as json_file:
    metadata = json.load(json_file)
    
src = os.path.join('model_ws', 'gsm_metadata.json')
with open(src, 'r') as f:
    gsm_metadata = json.load(f)   
    
from argparse import Namespace
meta = Namespace(**gsm_metadata)

## Use General Simulation Model to calculate TTD

Read MODFLOW model and create RTD object

In [5]:
print('Reading model information')

ml = fp.mf6.MFSimulation.load(sim_name='mfsim.nam', version='mf6', exe_name=metadata['modflow_path'],
                              sim_ws='optimal_model', strict=True, verbosity_level=0, load_only=None, verify_data=False)
model = ml.get_model()
rtd = rtd_ut.RTD_util(ml, 'flow', 'rt')
print('   ... done')

Reading model information
   ... done


Read model output and compute net inflow to drain cells

In [9]:
# read shapefile created in step 1--NHD flowlines intersected with model grid
src = os.path.join('gis', 'drain_segments.shp')
shp = gpd.read_file(src)

# read shapefile created in step 1--NHD flowlines intersected with model grid
src = os.path.join('gis', 'nhd_clip.shp')
nhd = gpd.read_file(src)
nhd_crs = nhd.crs

# read shapefile created in step 1--NHD flowlines intersected with model grid
domain = gpd.read_file(metadata['domain_name'])

# read enhanced model_grid file in model_ws
src = os.path.join('gis', 'model_grid.csv')
data = pd.read_csv(src)

# extract the drain budget terms from modflow output
rtd.get_budget('DRN')
drains = rtd.budget

# create a dataframe of drain flows
drn_df = pd.DataFrame(drains[0])
drn_df['node'] = drn_df['node'] - 1

# merge drain segments (by model cells) with drain flows
shp_drn_df = shp.merge(drn_df, left_on='node', right_on='node', how='outer')
shp_drn_df = shp_drn_df[shp_drn_df.q < 0]

# save shapefile to model_ws
dst = os.path.join('optimal_model', 'drain_flows.shp')
shp_drn_df.to_file(dst)

# flow = drn_df.q.sum()

# particles_per_flow = total_number_of_particles / flow

# # make particle locations
# x_partloc, y_partloc, node_list, label_list = rtd.make_stream_particle_array(
#     shp_drn_df, data, particles_per_flow)

Read endpoint information

In [12]:
endpointfile = '{}_flow_rt_mod.mpend'.format(metadata['HUC8_name'])
ep_data = pd.read_csv(os.path.join('optimal_model', endpointfile))

The following line is necessary because of a bug in Flopy 3.3.2.  Hopefully the bug will be fixed in future versions. 

In [13]:
ep_data['Particle ID'] = ep_data['Particle ID'] - 1

In [14]:
def meantt(x):
    return np.mean(x)

def mediantt(x):
    return np.median(x)

def fracyoung(x):
    return (x < 65).sum() / x.shape[0]

def meanyoung(x):
    return x[x < 65].mean()

def medianold(x):
    return np.median(x[x >= 65])
    
def meanpath(x):
    return np.mean(x)

agg_func = {'rt': [meantt, mediantt, fracyoung, meanyoung , medianold], 'xyz_path_len': meanpath} 

In [37]:
nhd['Particle ID'] = nhd.NHDPlusID.astype(np.int64()).astype(str).str[-9:].astype(np.int32())
summary = ep_data.groupby('Particle ID').agg(agg_func)
nhd_age = summary.merge(nhd, left_index=True, right_on='Particle ID')

C:\Users\jjstarn\Miniconda3\envs\genmod20\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [38]:
rep_str = {('rt', 'meantt'): 'meantt', ('rt', 'mediantt'): 'mediantt',
 ('rt', 'fracyoung'): 'fracyoung', ('rt', 'meanyoung'): 'meanyoung', 
 ('rt', 'medianold'): 'medianold', ('xyz_path_len', 'meanpath'): 'meanpath',
 'maxft': 'maxstage', 'minft': 'minstage'}

In [39]:
nhd_age.rename(columns=rep_str, inplace=True)

In [41]:
nhd_age.set_index('Particle ID', inplace=True)

In [43]:
nhd_age = gpd.GeoDataFrame(nhd_age[['meantt', 'mediantt', 'fracyoung', 'meanyoung', 'medianold', 'meanpath',
       'StreamOrde',
       'maxstage', 'minstage', 
       'geometry']])
nhd_age.crs = nhd_crs

Uncomment the following the first time this notebook is run. It takes about 24 minutes to run for the 1000m Schuylkill grid.

In [46]:
model_ws = 'optimal_model'

In [ ]:
comid_dict = dict()

for comid, _df in ep_data.groupby('Particle ID'):
    t = _df.rt
    t.values.sort()
    n = t.shape[0]
    tt_cdf = np.linspace(1. / n, 1., n, endpoint=True)
    tmp = rtd.fit_dists(tt_cdf, t, [ss.weibull_min], fit_one=True, fit_two=True)
    comid_dict[comid] = tmp

In [47]:
dst = os.path.join(model_ws, 'comid_dict.pkl')
with open(dst, 'wb') as f:
    pickle.dump(comid_dict, f)

In [48]:
dst = os.path.join('optimal_model', 'comid_dict.pkl')
with open(dst, 'rb') as f:
    comid_dict = pickle.load(f)

In [49]:
li = ['she', 'loe', 'sce', 'shl', 'lol', 'scl', 'f']
df = pd.DataFrame()
x = np.linspace(0, 10000, 10000)

for key, value in comid_dict.items():
    rt = value['tt']['rt']
    num_values = rt.shape[0]
    
    pars = value['par']['two_weibull_min']
    nhd_age.loc[key, li] = pars
    
#     w1 = ss.weibull_min(*pars[0:3])
#     w2 = ss.weibull_min(*pars[3:6])
#     pdf = (pars[6]) * w1.pdf(x) + (1-pars[6]) * w2.pdf(x)


In [50]:
dst = os.path.join('optimal_model', 'nhd_age.shp')
nhd_age.to_file(dst)

C:\Users\jjstarn\Miniconda3\envs\genmod20\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [51]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

var = 'fracyoung'
dum = nhd.plot(ax=ax, linewidth=0.75, color='cornflowerblue')
dum = gpd.GeoDataFrame(nhd_age).plot(column=var, legend=False, ax=ax, cmap=plt.cm.nipy_spectral, linewidth=1)
dum = domain.plot(ax=ax, color='none', edgecolor='black')
vmin=0
vmax=1
sm = plt.cm.ScalarMappable(cmap='nipy_spectral', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
cbaxes = fig.add_axes([0.50, 0.85, 0.3, 0.025]) 
cb = fig.colorbar(sm, ax=ax, cax=cbaxes, orientation='horizontal')  
ax.set_aspect(1)
dum = fig.suptitle('Fraction of young water')
# fig.set_tight_layout(True)

dst = os.path.join('optimal_model', 'metric_maps_frac.png')
plt.savefig(dst)

<IPython.core.display.Javascript object>

In [52]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

var = 'meanyoung'
dum = nhd.plot(ax=ax, linewidth=0.75, color='cornflowerblue')
dum = gpd.GeoDataFrame(nhd_age).plot(column=var, legend=False, ax=ax, cmap=plt.cm.nipy_spectral, linewidth=1)
dum = domain.plot(ax=ax, color='none', edgecolor='black')
vmin=0
vmax=65
sm = plt.cm.ScalarMappable(cmap='nipy_spectral', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
cbaxes = fig.add_axes([0.50, 0.85, 0.3, 0.025]) 
cb = fig.colorbar(sm, ax=ax, cax=cbaxes, orientation='horizontal')  
ax.set_aspect(1)
dum = fig.suptitle('Mean age of young water')
# fig.set_tight_layout(True)

dst = os.path.join('optimal_model', 'metric_maps_ageyoung.png')
plt.savefig(dst)

<IPython.core.display.Javascript object>

In [53]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

var = 'medianold'
dum = nhd.plot(ax=ax, linewidth=0.75, color='cornflowerblue')
dum = gpd.GeoDataFrame(nhd_age).plot(column=var, legend=False, ax=ax, cmap=plt.cm.nipy_spectral, linewidth=1)
dum = domain.plot(ax=ax, color='none', edgecolor='black')
vmin=65
vmax=nhd_age[var].max()
sm = plt.cm.ScalarMappable(cmap='nipy_spectral', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
cbaxes = fig.add_axes([0.50, 0.85, 0.3, 0.025]) 
cb = fig.colorbar(sm, ax=ax, cax=cbaxes, orientation='horizontal')  
ax.set_aspect(1)
dum = fig.suptitle('Median age of old water')
# fig.set_tight_layout(True)

dst = os.path.join('optimal_model', 'metric_maps_medianold.png')
plt.savefig(dst)

<IPython.core.display.Javascript object>

In [54]:
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)

var = 'meanpath'
dum = nhd.plot(ax=ax, linewidth=0.75, color='cornflowerblue')
dum = gpd.GeoDataFrame(nhd_age).plot(column=var, legend=False, ax=ax, cmap=plt.cm.nipy_spectral, linewidth=1)
dum = domain.plot(ax=ax, color='none', edgecolor='black')
vmin=0
vmax=nhd_age[var].max()
sm = plt.cm.ScalarMappable(cmap='nipy_spectral', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# fake up the array of the scalar mappable. Urgh...
sm._A = []
cbaxes = fig.add_axes([0.50, 0.85, 0.3, 0.025]) 
cb = fig.colorbar(sm, ax=ax, cax=cbaxes, orientation='horizontal')  
ax.set_aspect(1)
dum = fig.suptitle('Mean path length')
# fig.set_tight_layout(True)

dst = os.path.join('optimal_model', 'metric_maps_meanpath.png')
plt.savefig(dst)

<IPython.core.display.Javascript object>

In [55]:
nhd_age_df = pd.DataFrame(nhd_age)

fig, ax = plt.subplots(2, 2, sharex=True)
dum = nhd_age_df.groupby('StreamOrde').median().plot(kind='bar', y='fracyoung', ax=ax[0,0], legend=False)
dum = ax[0,0].set_ylabel('Fraction young water')

dum = nhd_age_df.groupby('StreamOrde').median().plot(kind='bar', y='meanyoung', ax=ax[0,1], legend=False)
dum = ax[0, 1].set_ylabel('Mean age of young water')

dum = nhd_age_df.groupby('StreamOrde').median().plot(kind='bar', y='medianold', ax=ax[1,0], legend=False)
dum = ax[1,0].set_xlabel('Stream order')
dum = ax[1,0].set_ylabel('Median age of old water')

dum = nhd_age_df.groupby('StreamOrde').median().plot(kind='bar', y='meanpath', ax=ax[1,1], legend=False)
dum = ax[1,1].set_xlabel('Stream order')
dum = ax[1,1].set_ylabel('Mean path length')

fig.set_tight_layout(True)

dst = os.path.join('optimal_model', 'bar_charts.png')
plt.savefig(dst)
# for i, label in enumerate(list(df.index)):
#     score = df.ix[label]['Score']
#     ax.annotate(str(score), (i, score + 0.2))

<IPython.core.display.Javascript object>

Extract recharge volume from subwatersheds and use it to convert loads to concentrations

In [56]:
# AtWtNO3 = 62.004 #g·mol−1
# AtWtN = 14.007 

# convertN = AtWtN ** -1 * AtWtNO3
# convertN

In [57]:
# tomorrow_load = 'total_04080798'
# waupaca_load = 'total_04081000'
# tomorrow_C = 'C_{}'.format(tomorrow_load)
# waupaca_C = 'C_{}'.format(waupaca_load)

# # read in any array to use as the template
# src = os.path.join(model_dir, 'reach_int.tif')

# ras = gmu.SourceProcessing()
# ras.read_raster(src)
# # ras.plot_raster(which_raster='new')

# # process the Tomorrow River watershed onto that raster
# src = os.path.join(gis_dir, 'watersheds', 'tom_area.shp')
# ras.process_vector_data(src, 'active')

# # recharge to total Tomorrow River watershed in m^3 / d
# RTom = (recharge[0, ...] * ras.new_array).sum()

# # recharge in m^3 / s
# RTom_ms = RTom * 86400

# # convert m^3 to L
# RTom = RTom * 10 ** 3

# # calculate recharge per year from per day
# RTom = RTom * 365.25

# # number of cells in watershed times the area of the cell
# ATom = ras.new_array.sum() * 304.8 * 304.8 

# # convert mass N to mass NO3
# # mass[tomorrow_load] = mass[tomorrow_load] * convertN

# # convert mass NO3 in kg to mg
# mass[tomorrow_load] = mass[tomorrow_load]  * 10 ** 6 

# # calculate concentration based on yearly volume of recharge
# mass[tomorrow_C] = mass[tomorrow_load] / RTom 

In [58]:
# # read in any array to use as the template
# src = os.path.join(model_dir, 'reach_int.tif')

# ras = gmu.SourceProcessing()
# ras.read_raster(src)

# # process the Waupaca River watershed onto that raster
# src = os.path.join(gis_dir, 'watersheds', wau_area.shp')
# ras.process_vector_data(src, 'active')
# ras.plot_raster(which_raster='new')

# # recharge to total Tomorrow River watershed in m^3 / d
# RWau = (recharge[0, ...] * ras.new_array).sum()

# # recharge in m^3 / s
# RWau_ms = RWau * 86400

# # convert m^3 to L
# RWau = RWau * 10 ** 3

# # calculate recharge per year from per day
# RWau = RWau * 365.25

# # number of cells in watershed times the area of the cell
# AWau = ras.new_array.sum() * 304.8 * 304.8 

# # convert mass N to mass NO3
# mass[waupaca_load] = mass[waupaca_load] * convertN

# # convert mass NO3 in kg to mg
# mass[waupaca_load] = mass[waupaca_load]  * 10 ** 6 

# # calculate concentration based on yearly volume of recharge
# mass[waupaca_C] = mass[waupaca_load] / RWau 

Expand the concentration input history to include background values for a long time before the data set starts. 

In [59]:
# freq = 365.25
# freq_str = '{}D'.format(freq)
# dates = pd.date_range('1850-01-01', '2020-01-01',
#                       freq=freq_str) + pd.Timedelta(days=182.625)

# # create a union of the original index and the desired index
# t = mass.index.union(dates)
# # reindex the tracer df using the unioned index
# mass = mass.reindex(t)
# # fill in the gaps corresponding to the new dates using interpolation
# mass = mass.interpolate('slinear')

# mass.fillna(method='bfill', inplace=True)

Fit a logistic curve to the input data in order to extrapolate ahead in time to when we have some baseflow nitrate data.

In [60]:
# mass['date'] = mass.index
# mass['jdate'] = mass.index.to_julian_date()

# xdata = mass['jdate'].loc[first_data:last_data]
# ydata = mass[tomorrow_C].loc[first_data:last_data]

# def sigmoid(x, L , x0, k, b):
#     y = L / (1 + np.exp(-k * (x - x0))) + b
#     return (y)

# p0 = [max(ydata), np.median(xdata), 0.00001, min(ydata)] # this is an mandatory initial guess

# popt, pcov = so.curve_fit(sigmoid, xdata, ydata, p0, method='dogbox')    

# yhat = sigmoid(mass.jdate, *popt)

# mass.loc[last_data:, tomorrow_C] = yhat

# fig, ax = plt.subplots(1, 1, figsize=(6, 3))
# dum = ax.plot(mass.date.loc[first_data:last_data], ydata)
# dum = ax.plot(mass.date.loc[first_data:], yhat[first_data:])

Run the convolution to get predicted breakthrough

In [61]:
# mass['btc'] = np.convolve(mass[tomorrow_C], pdf_df['pmf'], mode='full')[:232]

Read the nitrogen in baseflow data from Matt Miller

In [62]:
# gnl = pd.read_excel('../data/Streamflow QW analysis/Tomororw River Slowflow and Baseflow.xlsx', index_col='Date', 
#                     usecols='A, B, C, E, F, H, I', parse_dates=True)

# feacols = ['Slowflow GW (m^3/s)', 'Baseflow (m^3/s)', 'Slowflow GW Nitrate (mg/L)',
#        'Baseflow Nitrate (mg/L)', 'Slowflow GW Load (kg/d)',
#        'Baseflow Load (kg/d)']

# greeknames = [r'Slowflow $({m}^3/s)$', r'Baseflow $({m}^3/s)$', r'Slowflow $NO_3$ $(mg/L)$',
#        'Baseflow $NO_3$ $(mg/L)$', 'Slowflow $NO_3$ $(kg/d)$',
#        'Baseflow $NO_3$ $(kg/d)$']

# newcolumns = dict(zip(feacols, greeknames))

# gnl = gnl.rename(columns=newcolumns)

Compute a couple different leaching fractions

In [63]:
# lo = 0.12
# hi = 0.15

# mass['btc.lo'] = mass.btc * lo
# mass['btc.hi'] = mass.btc * hi

# lw = 1
# fig, ax = plt.subplots(1, 1, figsize=(6, 4))
# date = gnl.index
# d = dt.datetime(2014, 1, 1)

# ax.set_title('Baseflow $NO_3$ $(mg/L)$')

# # ax.plot(date, gnl[r'Slowflow $NO_3$ $(mg/L)$'], lw=lw, label='slowflow')
# ax.plot(date, gnl[r'Baseflow $NO_3$ $(mg/L)$'], lw=lw, label='Measured')
# ax.plot(mass.date.loc[first_data:], mass.loc[first_data:, 'btc.lo'], label='Simulated (f = {:0.3f})'.format(lo))
# ax.plot(mass.date.loc[first_data:], mass.loc[first_data:, 'btc.hi'], label='Simulated (f = {:0.3f})'.format(hi))

# ax.set_xlabel('')
# ax.set_ylabel('$[NO_3]$ in $mg/L$')
# ax.legend(frameon=False)

# fig.set_tight_layout(True);
# dst = os.path.join(fig_dir, 'Tom_btc_porosity={} mean_age={:0.0f}.png'.format(por, df.rt.mean()))
# plt.savefig(dst)

In [64]:
# mass.to_csv('../figures/N_breakthrough.csv')

In [65]:
# np.median((.228,.087,.328,.1,.094,.053,.033,.2))
# np.mean((.228,.087,.328,.1,.094,.053,.033,.2))

# TODO
* try refining the model grid
* Use spline function to extend tracer signal
* Run and compare sample data
* Use tritium to estimate porosity
* Use NO$^3$ to estimate leaching fraction
* Do separately for Tomorrow and Waupaca
* Map Weibull parameters by catchment

In [66]:
# read the cell binary budget file (cbb)
# fname = os.path.join(model_ws, '{}.cbb'.format(model_name))
# cbb = fp.utils.CellBudgetFile(fname, precision='double')

# rtd.get_watertable()
# rtd.get_heads()

# drains = rtd.bud_obj.get_data(text='DRN', kstpkper=rtd.kstpkper)[0]
# recharge = rtd.bud_obj.get_data(text='RCH', kstpkper=rtd.kstpkper)[0]



# src = os.path.join(geo_ws, 'model_grid.csv')
# data = pd.read_csv(src)

# data['watertable'] = rtd.water_table.ravel()

# data.loc[drains.node - 1, 'drain_q'] = drains.q
# data['drain_q'].fillna(0, inplace=True)

# data.loc[recharge.node - 1, 'recharge_q'] = recharge.q
# data['recharge_q'].fillna(0, inplace=True)

# data['net_q'] = data['recharge_q'] + data['drain_q']
# data['net_q'].fillna(0, inplace=True)

# data['net_index'] = data['net_q'] < 0
# data['drain_index'] = data['drain_q'] < 0
# # if using drain_q set a minimum travel time for particles that discharge within their initial cell 


# net_or_drain = 'drain'

# index = '{}_index'.format(net_or_drain)
# q = '{}_q'.format(net_or_drain)

# inflow_cells = data.loc[data[index], 'node'].astype( np.int32 )

# reachcode = data.loc[data[index], 'comid']

# f = number_of_particles_per_group / data[q].sum()
# flubber = data.loc[data[index], q]
# particles_per_cell = np.rint( flubber * f ).astype( np.int32 )

In [67]:
# t = rtd.ep_data.rt
# t.values.sort()

# # number of particles
# n = t.shape[0]
# tt_cdf = np.linspace(1. / n, 1., n, endpoint=True)

# Fit RTDs

# fit_dict = rtd.fit_dists(tt_cdf, t, [ss.weibull_min], fit_one=True, fit_two=True)

# cdf = pd.DataFrame(fit_dict['tt'])
# fit = pd.DataFrame(fit_dict['cdf'])

# cdf.shape
# fit.shape

# cdf.reset_index(inplace=True)
# fit.reset_index(inplace=True)

# cdf.head()
# fit.head()

# df = pd.concat([cdf, fit], axis=1)
# df.head()

# fit

# print('Mean age is {:0.0f} years'.format(df.rt.mean()))

# Create a dataframe containing the PDF and PMF

# pars = fit_dict['par']['two_weibull_min']

# w1 = ss.weibull_min(*pars[0:3])
# w2 = ss.weibull_min(*pars[3:6])

# x = np.arange(0, 100, 1)
# pdf = (pars[6]) * w1.pdf(x) * (1-pars[6]) * w2.pdf(x)

# pdf_df = pd.DataFrame({'x': x, 'pdf': pdf, 'pmf': pdf / pdf.sum()})

# Plot the one and two components Weibull distributions

# fig, ax = plt.subplots(1, 2, figsize=(8, 4))

# dum = ax[0].plot(df.rt, df.rt_cdf, label='particles', color=KS5)
# dum = ax[0].plot(df.rt, df.one_weibull_min, label='one-component', color=KS5, ls='dashed')
# dum = ax[0].plot(df.rt, df.two_weibull_min, label='two-component', color=KS2)

# dum = ax[0].set_xscale('log')
# dum = ax[0].set_xlabel('Travel time, in years')
# dum = ax[0].set_ylabel('Cumulative frequency')
# dum = ax[0].legend(frameon=False, loc='lower right', title='TTD')


# dum = ax[1].hist(df.rt * por / 0.1, bins=1000, label='0.10', alpha=1.0, density=True, color=KS1)
# dum = ax[1].hist(df.rt * por / 0.2, bins=1000, label='0.20', alpha=1.0, density=True, color=KS4)
# dum = ax[1].hist(df.rt * por / 0.3, bins=1000, label='0.30', alpha=0.6, density=True, color=KS2)

# dum = ax[1].set_xlabel('Travel time, in years')
# dum = ax[1].set_ylabel('Bin frequency')
# dum = ax[1].set_xlim(0, 50)
# dum = ax[1].legend(frameon=False, loc='lower right', title='Porosity')

# fig.set_tight_layout(True)

# dst = os.path.join(fig_dir, 'ttds.png')
# plt.savefig(dst)

In [68]:

ax = sns.jointplot(x='Initial global x', y='Initial global y', data=ep_data, kind='hex', cmap=plt.cm.nipy_spectral, mincnt=1)
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)  # shrink fig so cbar is visible
cbar_ax = ax.fig.add_axes([.85, .25, .05, .4])  # x, y, width, height
plt.colorbar(cax=cbar_ax)

<IPython.core.display.Javascript object>

In [69]:
ax = sns.jointplot(x='Final global x', y='Final global y', data=ep_data, kind='hex', cmap=plt.cm.nipy_spectral, mincnt=1)
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)  # shrink fig so cbar is visible
cbar_ax = ax.fig.add_axes([.85, .25, .05, .4])  # x, y, width, height
plt.colorbar(cax=cbar_ax)

<IPython.core.display.Javascript object>

In [70]:
num_ran = 20 

rlist = np.random.choice(list(comid_dict.keys()), num_ran)

plot_rows = np.ceil(num_ran / 4).astype(np.int32())

fig, axs = plt.subplots(plot_rows, 4, figsize=(8, num_ran / 2))
ax = axs.ravel()

for plot, i in enumerate(rlist):

    tt = pd.DataFrame(comid_dict[i]['tt'])
    dum = ax[plot].plot(tt.rt, tt.rt_cdf, label='particles', color=KS5)
    dum = ax[plot].axvline(tt.rt.mean())
    maxx = tt.rt.max()
    x = np.linspace(0, maxx, 10000)
    
    try:
        fit = pd.DataFrame(comid_dict[i]['cdf'])
        
        pars = comid_dict[i]['par']['one_weibull_min']
        w1 = ss.weibull_min(*pars[0:3])

        cdf = w1.cdf(x)
        dum = ax[plot].plot(x, cdf, label='one-component', color=KS2)

        pars = comid_dict[i]['par']['two_weibull_min']
        w1 = ss.weibull_min(*pars[0:3])
        w2 = ss.weibull_min(*pars[3:6])

        cdf = (pars[6]) * w1.cdf(x) + (1-pars[6]) * w2.cdf(x)
        dum = ax[plot].plot(x, cdf, label='two-component', color=KS4)
        
        print('it worked!')
        
    except:
        'nope'

    dum = ax[1].legend(frameon=False, loc='lower right', title='Porosity')

    fig.set_tight_layout(True)

    dst = os.path.join(model_ws, 'ttds.png')
    plt.savefig(dst)

<IPython.core.display.Javascript object>

No handles with labels found to put in legend.


it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
it worked!
